# AutoRAG - DataCreation Pipeline

# Module

In [6]:
import warnings
warnings.filterwarnings(action='ignore')
import os
import sys
import dotenv
import nest_asyncio
dotenv.load_dotenv()
# nest_asyncio 적용
nest_asyncio.apply()

import pandas as pd
from glob import glob

# AutoRAG
from autorag.parser import Parser
from autorag.chunker import Chunker
from autorag.data.qa.schema import Raw, Corpus
from autorag.data.qa.filter.dontknow import dontknow_filter_rule_based
from autorag.data.qa.generation_gt.llama_index_gen_gt import make_basic_gen_gt,make_concise_gen_gt
from autorag.data.qa.query.llama_gen_query import factoid_query_gen
from autorag.data.qa.sample import random_single_hop

# llamaindex
from llama_index.llms.openai import OpenAI

# RawData Parse

In [55]:
file_paths = glob("../data/nursing/*")

In [48]:
idx = 16
print(file_paths[idx])

/home/livin/rag_pipeline/AutoRAG/data/nursing/14장 호흡기장애 대상자 간호.pdf


In [112]:
# parser 정의
parser = Parser(data_path_glob="/home/livin/rag_pipeline/AutoRAG/data/test/*", project_dir="../results/parse/test")
parser.start_parsing("../configs/Parse.yaml")

[10/11/24 16:50:50] INFO     [parser.py:33] >> Parsing Start...                                        ]8;id=240537;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/parser.py\parser.py]8;;\:]8;id=599190;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/parser.py#33\33]8;;\

                    INFO     [base.py:19] >> Running parser - langchain_parse module...                  ]8;id=656307;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/data/parse/base.py\base.py]8;;\:]8;id=987080;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/data/parse/base.py#19\19]8;;\

KeyError: 'upstagelayoutanalysis'

In [109]:
a = pd.read_parquet(f"../results/parse/test/4/0.parquet")

In [111]:
a["texts"][0]

"<header id='0' style='font-size:22px'>3</header><p id='1' data-category='paragraph' style='font-size:14px'>• 오디오 등을 장착하거나 차량 전면부에 판금<br>이나 도장을 하는 경우에 안전벨트 프리텐셔<br>너 작동에 영향을 미칠 수 있습니다.</p><br><p id='2' data-category='paragraph' style='font-size:14px'>이럴 경우 당사 직영 하이테크센터나 블루핸<br>즈에서 상담을 받으십시오. 안전벨트 프리텐<br>셔너가 제대로 작동하지 않으면, 에어백 경<br>고등(=)이 켜집니다.</p><br><h1 id='3' style='font-size:14px'>• 다음의 상황이 발생하면 당사 직영 하이테크<br>센터나 블루핸즈에서 신속하게 점검을 받으<br>십시오.</h1><br><p id='4' data-category='list' style='font-size:14px'>- 시동 'ON' 했을 때 에어백 경고등이 켜지<br>지 않는 경우<br>- 에어백 경고등이 시동 'ON' 했을 때 3-6초<br>이후에도 계속 켜져 있는 경우<br>- 시동 'ON' 했을 때 에어백 경고등이 깜빡<br>이는 경우<br>- 주행 중에 에어백 경고등이 켜질 경우</p><h1 id='5' style='font-size:18px'>(cid:1155)(cid:1162)</h1><br><p id='6' data-category='list' style='font-size:14px'>• 안전벨트 프리텐셔너가 작동하면, 큰 소음이<br>날 뿐만 아니라 연기 같은 미세한 가루가 보<br>일 수 있습니다. 이 가루는 독성이 없는 가스<br>입자이지만 피부에 자극을 줄 수도 있습니<br>다. 가루를 들이마시지 말고 손과 얼굴을 깨<br>끗이 닦으십시오.<br>• 안전벨트 프리텐셔너는 작동할 때 뜨거운 열<br>을 내므로 작동한 후 몇 분 동안은 안전벨트<br>프리텐셔너를 

In [81]:
# Integration
file_paths = glob("../data/nursing/*")
file_paths = [[i, n] for n,i in enumerate(file_paths)]
file_paths = sorted(file_paths, key=lambda x: int(x[0].split("/")[-1][:2]))

# nursing 폴더에 있는 parquet을 하나씩 읽어서 통합 parquet만들기

df = pd.DataFrame()
for file_path in file_paths:
    df = pd.concat([df, pd.read_parquet(f"../results/parse/nursing/{file_path[1]}/0.parquet")])
    # print(file_path[0])

df.index = range(len(df))
# df.to_parquet("../results/parse/nursing/nursing.parquet")


In [89]:
parse_df = pd.read_parquet("../results/parse/nursing/nursing.parquet")
parse_df

,texts,path,page,last_modified_datetime
0,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
1,<h1 id='0' style='font-size:18px'>강의용 자료파일 사용 ...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
2,<h1 id='0' style='font-size:18px'>강의용 자료파일 사용 ...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
3,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
4,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
5,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
6,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
7,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
8,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
9,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11


# Chunking

In [90]:
chunker = Chunker.from_parquet(parsed_data_path="../results/parse/nursing/nursing.parquet", project_dir="../results/chunk/nursing")
chunker.start_chunking("../configs/Chunk.yaml")

[10/11/24 16:30:02] INFO     [chunker.py:48] >> Chunking Start...                                     ]8;id=703754;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py\chunker.py]8;;\:]8;id=547100;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py#48\48]8;;\

                    INFO     [base.py:22] >> Running chunker - llama_index_chunk module...               ]8;id=398431;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/data/chunk/base.py\base.py]8;;\:]8;id=19214;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/data/chunk/base.py#22\22]8;;\

                    INFO     [chunker.py:55] >> Chunking Done!                                        ]8;id=264152;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py\chunker.py]8;;\:]8;id=835233;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py#55\55]8;;\

In [91]:
# Chunk 결과 확인
corpus_df = pd.read_parquet("../results/chunk/nursing/1/0.parquet")
corpus_df

,doc_id,contents,path,start_end_idx,metadata
0,1df6e763-a6ec-4cc9-9083-41968682eb59,파일 제목: 00 도입부(하권).pdf\n 내용: <p id='0' d...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,"[0, 2796]","{'last_modified_datetime': '2024-10-11', 'next..."
1,e68717ac-554d-4e44-b6c7-bc13e8089921,파일 제목: 00 도입부(하권).pdf\n 내용: id='24' dat...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,"[2214, 3014]","{'last_modified_datetime': '2024-10-11', 'next..."
2,f63b4bb3-7b69-4e1d-a3a2-879111de697c,파일 제목: 14장 호흡기장애 대상자 간호.pdf\n ...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,"[0, 2450]","{'last_modified_datetime': '2024-10-11', 'next..."
3,d2c85275-9344-4a74-91eb-c24ac10ef99d,파일 제목: 14장 호흡기장애 대상자 간호.pdf\n ...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,"[1867, 4611]","{'last_modified_datetime': '2024-10-11', 'next..."
4,e1fc5873-d1a6-48aa-9452-3b03eb16b438,파일 제목: 14장 호흡기장애 대상자 간호.pdf\n ...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,"[4000, 6752]","{'last_modified_datetime': '2024-10-11', 'next..."
...,...,...,...,...,...
242,6b739c72-76e8-467d-aa01-33ac35f25169,파일 제목: 26장 피부장애 대상자 간호.pdf\n 내용:...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,"[2132, 4860]","{'last_modified_datetime': '2024-10-11', 'next..."
243,b5cd7d04-2c5a-410b-90ad-d2af9116be03,파일 제목: 26장 피부장애 대상자 간호.pdf\n 내용:...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,"[4285, 7007]","{'last_modified_datetime': '2024-10-11', 'next..."
244,c2800e70-b69b-4c65-aa7a-e76f14bfa092,파일 제목: 26장 피부장애 대상자 간호.pdf\n 내용:...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,"[6413, 7331]","{'last_modified_datetime': '2024-10-11', 'next..."
245,60ba4dac-fc52-4775-b1a2-6b882702cd59,파일 제목: 27장 화상간호.pdf\n 내용: <p id='0' da...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,"[0, 2823]","{'last_modified_datetime': '2024-10-11', 'next..."


In [102]:
corpus_df.iloc[2]

doc_id                        f63b4bb3-7b69-4e1d-a3a2-879111de697c
contents         파일 제목: 14장 호흡기장애 대상자 간호.pdf\n ...
path             /home/livin/rag_pipeline/AutoRAG/data/nursing/...
start_end_idx                                            [0, 2450]
metadata         {'last_modified_datetime': '2024-10-11', 'next...
Name: 2, dtype: object

# QA Creation

In [3]:
# load RAW data
raw_df = pd.read_parquet("../results/chunk/hyundai_upstage/0.parquet")
raw_instance = Raw(raw_df)

# load Corpus data
corpus_df = pd.read_parquet("../results/chunk/hyundai_upstage/0.parquet")
corpus_instance = Corpus(corpus_df, raw_instance)

In [12]:
# llm 선언
llm = OpenAI(model="gpt-4o-mini")

# QA 데이터 생성 파이프라인
initial_qa = (
    corpus_instance.sample(random_single_hop, random_state=4, n=30) # 몇개의 corpus를 뽑을지 결정
    .map(
        lambda df: df.reset_index(drop=True), # Drop index
    )
    .make_retrieval_gt_contents() # retrieval_gt_contents 생성
    .batch_apply(
        factoid_query_gen,  # query generation
        llm=llm,
        lang="ko",
    )
    .batch_apply(
        make_basic_gen_gt,  # answer generation (basic)
        llm=llm,
        lang="ko",
    )
    .filter(
        dontknow_filter_rule_based,  # unanswerable filter
        lang="ko",
    )
)

[10/08/24 17:03:48] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=624774;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=181230;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[10/08/24 17:03:49] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=870121;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=354221;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=250077;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=755106;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=794991;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=26241;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=624022;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=567964;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

...

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=517817;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=736670;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=717684;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=633212;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[10/08/24 17:03:54] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=355869;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=656520;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[10/08/24 17:03:55] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=571567;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=836383;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[10/08/24 17:03:56] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=522397;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=748958;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [13]:
# 생성된 qa 저장
initial_qa.to_parquet('../results/qa/hyundai_upstage_qa_4.parquet', '../results/qa/hyundai_upstage_corpus_4.parquet')

In [15]:
# 생성된 qa 확인
initial_qa.data.head(2)

,qid,retrieval_gt,retrieval_gt_contents,query,generation_gt
0,98243a5a-d2c6-4b0f-8a49-295cf8f1c011,[[8e6d83ba-f2b0-4c4b-9707-808b89507ac8]],[[파일 제목: hyndai.pdf\n 내용: id='98' data-categor...,견인 후크를 설치하는 방법은 무엇인가요?,[견인 후크를 설치하는 방법은 다음과 같습니다:\n\n1. 화물칸 공구 박스 내에 ...
1,5433ec98-6e54-4781-a9f2-535c3ad02e4e,[[ce3ff8fa-c629-4da6-8a11-99e6d7e1220a]],[[파일 제목: hyndai.pdf\n 내용: 분사하지 마십시오.<br>• 전방 레...,전방 충돌방지 보조 기능을 설정하려면 어떤 절차를 따라야 합니까?,[전방 충돌방지 보조 기능을 설정하려면 다음 절차를 따라야 합니다:\n\n1. 시동...


In [16]:
# Chunk 결과 확인
corpus_df = pd.read_parquet("/home/livin/rag_pipeline/AutoRAG/results/qa/hyundai_upstage_qa_3.parquet")
corpus_df.head(2)

,qid,query,retrieval_gt,generation_gt
0,052b323b-fa7f-41f0-9dd6-41791d2a5d6f,에어컨을 사용한 후 차량 밑바닥에서 물이 떨어지는 이유는 무엇인가요?,[[5d54c7a1-134d-4c29-a07f-68f68d7979a8]],[에어컨을 사용한 후 차량 밑바닥에서 물이 떨어지는 것은 정상적인 현상입니다.]
1,3e372364-e950-41bb-beff-dad5cecfdedd,"USB 충전 단자는 어떤 용도로 사용되며, 이 단자를 통해 미디어를 재생할 수 있습니까?",[[596ccf0f-1e57-41dc-908a-a3f99803ba80]],"[USB 충전 단자는 충전 전용으로 사용되며, 이 단자를 통해서는 인포테인먼트 시스..."


# QA Integration
* 다양한 QA 셋을 통합하여 하나의 QA 셋으로 만듦

In [17]:
qa_list = ["../results/qa/hyundai_upstage_qa_1.parquet", "../results/qa/hyundai_upstage_qa_2.parquet", "../results/qa/hyundai_upstage_qa_3.parquet", "../results/qa/hyundai_upstage_qa_4.parquet"]

In [19]:
# 여러 qa 파일을 하나의 qa 파일로 합치기
qa_df = pd.concat([pd.read_parquet(qa) for qa in qa_list], ignore_index=True)

# qa 저장
qa_df.to_parquet("../results/qa/hyundai_upstage_qa_1234.parquet")

# Test

In [20]:
qa_df = pd.read_parquet('../results/qa/hyundai_upstage_qa_1234.parquet')
corpus_df = pd.read_parquet('../results/qa/hyundai_upstage_corpus.parquet')

In [21]:
qa_df.head(2)

,qid,query,retrieval_gt,generation_gt
0,f81a5a16-1c6b-4e59-af51-6e4b0959b456,고속도로 주행 보조 설정은 몇 페이지에 나와 있습니까?,[[937e0eea-65b2-4195-b374-a72c6eccb872]],[고속도로 주행 보조 설정은 7-81 페이지에 나와 있습니다.]
1,894f74ee-f5e1-41d8-a5ef-484b50f28bb0,디지털 센터 미러의 올바른 사용에 대한 경고 및 지침을 따르지 않으면 어떤 위험이 ...,[[05115da3-694b-49c1-9004-c59462259054]],[디지털 센터 미러의 올바른 사용에 대한 경고 및 지침을 따르지 않으면 심각한 사고...


In [23]:
len(qa_df)

120

In [24]:
corpus_df.head(2)

,doc_id,contents,metadata
0,af0d9d7f-a1ca-40a9-9c16-67676f0bc620,파일 제목: hyndai.pdf\n 내용: <h1 id='0' style='font...,"{'last_modified_datetime': '2024-09-06', 'next..."
1,d982d5c6-41fb-4d96-ad9a-76929fe018b5,파일 제목: hyndai.pdf\n 내용: 포 함 하 여 설 명<br>하 고 있 습...,"{'last_modified_datetime': '2024-09-06', 'next..."


In [25]:
idx = 0
print(qa_df.loc[idx, 'query'])
print(qa_df.loc[idx, 'generation_gt'])

고속도로 주행 보조 설정은 몇 페이지에 나와 있습니까?
['고속도로 주행 보조 설정은 7-81 페이지에 나와 있습니다.']


In [26]:
corpus_df[corpus_df["doc_id"] == qa_df.loc[idx, "retrieval_gt"][0][0]]["contents"].item()

"파일 제목: hyndai.pdf\n 내용: 해제 기능 5-30<br>고속도로 주행 보조 설정 7-81<br>전자식 차일드 락 5-30<br>고속도로 주행 보조 이상 및 제한 사항 7-88<br>차 밖에서의 잠금/잠금 해제 5-26<br>고속도로 주행 보조 작동 7-82<br>차 안에서의 잠금/잠금 해제 5-28<br>공조 장치용 에어필터 9-20<br>차속 감응 자동 도어 잠금 장치 5-29<br>에어필터 교체 9-20<br>충돌 감지 자동 도어 잠금 해제 장치 5-29<br>에어필터 점검 9-20<br>도어/테일게이트/<br>교차로나 건널목에서의 긴급 상황 대처 2-9<br>유리창을 열고 닫을 때 조심 2-8<br>규격 타이어 장착 및 타이어 공기압 수시 점검<br>듀얼 와이드 선루프 5-59<br>2-4<br>물체 끼임 인식 기능 5-61<br>기타 주의 사항 2-11<br>선루프 열림 경고 5-62<br>선루프 초기화 5-62<br>나 선블라인드 (앞)/파워 선블라인드 (뒤) 5-59<br>슬라이드 열림/닫힘 (앞좌석) 5-60<br>내 비게이션 기반 스마트 크루즈 컨트롤 (NSCC)<br>틸트 열림/닫힘 (앞좌석) 5-60<br>7-71<br>내비게이션 기반 스마트 크루즈 컨트롤 설정<br>7-72 마<br>내비게이션 기반 스마트 크루즈 컨트롤 작동<br>7-72 무 선 소프트웨어 업데이트 5-85<br>작동 제한 사항 7-74 소프트웨어 업데이트 승인 5-85<br>냉각수 점검 9-15 소프트웨어 자동 다운로드 5-85<br>냉각수량 점검 및 보충 9-15 미러 5-42<br>냉각수의 교체 9-17 실내 미러 5-42<br>실외 미러 5-48</p><footer id='14' style='font-size:18px'>I-2</footer><br><h1 id='15' style='font-size:14px'>MX5HEV_ko_KR.book Page 3</h1><p id='16' data-category='paragraph' style='font-siz